In [1]:
!pip install tensorflow
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests 
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.1 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 5.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import numpy as np
import random
import pickle as pkl
from typing import Any, Dict

import gym
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete

import torch
import torch.nn as nn

from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

%load_ext tensorboard

In [64]:
#python -m pip install --no-index --find-links=relative/dir/ SomePackage
!pip install --no-index --find-links=file://localhost/Users/emadhekar/work/tria/gym-examples gym-examples

#!pip install -e Users/emadhekar/work/tria/gym-examples
#!pip install -e gym-examples --find-links file://Users/emadhekar/work/tria

Looking in links: file://localhost/Users/emadhekar/work/tria/gym-examples
ERROR: Could not find a version that satisfies the requirement gym-examples (from versions: none)
ERROR: No matching distribution found for gym-examples


# tria environment base parameters 

In [65]:
env_name = 'TriaEnv'

t_ini= 70.0; h_ini= 40.0; a_ini= 10.0 

t_min =-40.0; t_max=110; h_min=0.0; h_max=100.0; a_min=0.0; a_max=5000.0

act_state = 2

stat_rand_min = -1.0; stat_rand_max = 1.0

equilibrium_cycles= 60

r1 = -0.25; r2 = -0.5; r3 = 2; nr3 = -2

const_weight_vec  = [1, 1, 1, 1]

d3 = {
     0 : [65.0, 80.0, 50.0, 85.0, 40.0, 90.0], 
     1 : [30.0, 50.0, 20.0, 60.0, 10.0, 70.0], 
     2 : [0.0, 19.0, 200.0, 599.0, 600.0, 2000.0]
    }

d1 = {0: [65.0, 80.0], 1: [30.0, 50.0], 2: [0.0, 20.0]}

ppo_model_timesteps= 20000; neural_model_timesteps=20000; a2c_model_timesteps=20000

ppo_model = env_name + 'ppo'; neural_model = env_name + 'ppo-neural'; a2c_model = env_name + 'a2c'


# tria hyper-parameter settings

In [66]:
n_trials = 100

n_jobs = 1

n_startup_trials = 5

n_evaluations = 2

n_timesteps = int(2e4)

eval_freq = int(n_timesteps / n_evaluations)

n_eval_envs = 2

n_eval_episodes = 5

timeout = int(60 * 15) 

delafult_hyperparams = {
    'policy' :  'MlpPolicy',
    'env' : env_name
}

# tria A2C model sampler for hyperparameters

In [67]:
def tria_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:

    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    
    gae_lambda = 1.0 - trial.suggest_float("gae_lambda", 0.001, 0.2, log=True)
    
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
    
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    
    ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
    
    ortho_init = trial.suggest_categorical("ortho_init", [False, True])
    
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    # 
    trial.set_user_attr("gamma_", gamma)
    
    trial.set_user_attr("gae_lambda_", gae_lambda)
    
    trial.set_user_attr("n_steps", n_steps)
    
    #neural network selection choice
    net_arch = [
        {"pi": [64], "vf": [64]}
        if net_arch == "tiny"
        else {"pi": [64, 64], "vf": [64, 64]}
    ]

    # activation / non-linearity selection 
    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]
    
    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "gae_lambda": gae_lambda,
        "learning_rate": learning_rate,
        "ent_coef": ent_coef,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
            "ortho_init": ortho_init,
        },
    }
    

# definition of callback used for evaluating and reporting trials

In [68]:
class TriaTrialEvalCallback(EvalCallback):
    def __init__( self, eval_env: VecEnv, trial: optuna.Trial, n_eval_episodes: int = 5, eval_freq: int = 10000, deterministic: bool = True, verbose: int = 0 ): 
        super().__init__(eval_env= eval_env, n_eval_episodes= n_eval_episodes, eval_freq= eval_freq, deterministic= deterministic, verbose= verbose)
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False
        
    def _on_step(self) -> bool:
        
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
            
        return True

In [69]:
from gym import utils
import logging

logger = logging.getLogger(__name__)

class TriaEnv(Env, utils.EzPickle):
    metadata = {'render.modes': ['human']}
    
    def __init__(self):
        self.action_space = MultiDiscrete(np.array([act_state,act_state,act_state,act_state,act_state]))
        
        self.observation_space = Box(low=np.array([t_min, h_min, a_min]), high=np.array([t_max, h_max, a_max]), dtype=np.float32)
        
        self.state = [t_ini + random.uniform(stat_rand_min, stat_rand_max), h_ini + random.uniform(stat_rand_min, stat_rand_max), a_ini + random.uniform(stat_rand_min, stat_rand_max)]
        
        #print('^^^', self.state, self.action_space)
        
        self.equilibrium_cycles_len = equilibrium_cycles
        
    def step(self, action):
        
        ap_scaled = [1 if e == 1 else -1 for e in action]  # 0 (off) => -1 and 1 (on) => 1
        
        actionPrime = [a * b for a, b in zip(ap_scaled, const_weight_vec)] 
        
        actionAlgo = [actionPrime[a] - actionPrime[len(actionPrime) -a -1] for a in range(len(actionPrime) // 2)]
        
        actionAlgo.append(actionPrime[len(actionPrime) // 2])                                                              
        
        #print('***',actionAlgo, self.state)
        
        self.state = [a + b for a, b in zip(actionAlgo, self.state)]
        
        #print('&&&', actionAlgo, self.state)
        
        #reduce tria simulation length by 1 second
        self.equilibrium_cycles_len -= 1
        
        reward = [r3 if e >= d3[i][0] and e<= d3[i][1] else r2 if e >= d3[i][2] and e<= d3[i][3] else r1 if e >= d3[i][4] and e <= d3[i][5] else nr3 for i, e in enumerate(self.state)]
        #reward = [r3 if e >= d1[i][0] and e <= d1[i][1] else nr3  for i, e in enumerate(self.state)]

        reward = sum(reward)
        #print('$$$', reward)
            
        if self.equilibrium_cycles_len <= 0:
            terminated = True
        else:
            terminated = False
            
        info = {}
        #print('reward:{} state:{}'.format(reward, self.state))
        return self.state, reward, terminated, False, info
    
    def render(self):
        pass
    
    def reset(self):
        
        self.state =[t_ini + random.uniform(stat_rand_min, stat_rand_max),h_ini + random.uniform(stat_rand_min, stat_rand_max),a_ini + random.uniform(stat_rand_min, stat_rand_max)]
        #print('@@@', self.state)
        self.equilibrium_cycles_len = equilibrium_cycles
        
        return self.state, {}
        

In [29]:
def objective(trial: optuna.Trial) -> float:

    kwargs = delafult_hyperparams.copy()
    # Tria hyperparameters
    kwargs.update(tria_a2c_params(trial))
    # Create the RL model
    model = A2C(**kwargs)
    # Create env used for evaluation
    eval_envs = TriaEnv(Env)#make_vec_env(env_name, n_eval_envs)
    # Create the callback that will periodically evaluate
    # and report the performance
    eval_callback = TrialEvalCallback(
        eval_envs,
        trial,
        n_eval_episodes= n_eval_episodes, #N_EVAL_EPISODES,
        eval_freq= eval_freq, #EVAL_FREQ,
        deterministic=True
    )
    nan_encountered = False
    
    try:
        model.learn(n_timesteps, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_envs.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward


In [30]:
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=n_startup_trials)
    # Do not prune before 1/3 of the max budget is used
    pruner = MedianPruner(
        n_startup_trials=n_startup_trials, n_warmup_steps=n_evaluations // 3
    )

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

    try:
        study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs, timeout=timeout)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print(f"  Value: {trial.value}")

    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print(f"    {key}: {value}")

    # Write report
    study.trials_dataframe().to_csv("study_results_a2c_cartpole.csv")

    with open("study.pkl", "wb+") as f:
        pkl.dump(study, f)

    fig1 = plot_optimization_history(study)
    fig2 = plot_param_importances(study)

    fig1.show()
    fig2.show()

[I 2023-03-13 20:24:56,384] A new study created in memory with name: no-name-ea939ddc-9dc6-4e29-afd3-146182146cb9
[W 2023-03-13 20:24:56,415] Trial 0 failed with parameters: {'gamma': 0.0006142605993311873, 'max_grad_norm': 0.5240056412177426, 'gae_lambda': 0.012710285175802274, 'exponent_n_steps': 10, 'lr': 0.25656875854694317, 'ent_coef': 0.06323517695014111, 'ortho_init': True, 'net_arch': 'tiny', 'activation_fn': 'relu'} because of the following error: NameNotFound("Environment TriaEnv doesn't exist. ").
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_142/4228427625.py", line 7, in objective
    model = A2C(**kwargs)
  File "/opt/conda/lib/python3.10/site-packages/stable_baselines3/a2c/a2c.py", line 84, in __init__
    super().__init__(
  File "/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py", line 75

NameNotFound: Environment TriaEnv doesn't exist. 

In [70]:
env= gym.make("TriaEnv")

print("1. Sample observation space: {}".format(env.observation_space.sample()))
print("2. Sample action space     : {}".format(env.action_space.sample()))
print("3. Sample state            : {}".format(env.state))

NameNotFound: Environment TriaEnv doesn't exist. 

In [206]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    #print(state)
    terminated = False
    score = 0 #[0,0,0] 
    
    while not terminated:
        #env.render()
        action = env.action_space.sample()
        #print(action, terminated , reward)
        #print(env.step(action))
        next_state, reward, terminated, truncated, info = env.step(action) 
        score += reward #[a + b for a, b in zip(reward, score)]
    print('Episode: {} Score: {}'.format(episode, score))
env.close()

Episode: 1 Score: 332.5
Episode: 2 Score: 243.0
Episode: 3 Score: 360
Episode: 4 Score: 118.0
Episode: 5 Score: 245.0


In [207]:
log_path = os.path.join('training','logs')
log_path

'training/logs'

In [208]:
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [209]:
model.learn(total_timesteps=ppo_model_timesteps)

Logging to training/logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 140  |
|    iterations      | 1    |
|    time_elapsed    | 14   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 86          |
|    iterations           | 2           |
|    time_elapsed         | 47          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011640093 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.46       |
|    explained_variance   | -0.00329    |
|    learning_rate        | 0.0003      |
|    loss                 | 1.31e+03    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0123     |
|    value_loss           | 2.79e+03    |
-----------------------------------------
---

In [210]:
tria_model_path = os.path.join('training','save', ppo_model)

In [211]:
model.save(tria_model_path)

In [212]:
del model

In [213]:
model = PPO.load(tria_model_path, env=env)

In [214]:
evaluate_policy(model, env, n_eval_episodes=20, render=True)

(360.0, 0.0)

In [183]:
env.close()

In [184]:
episodes=10
for episode in range(1, episodes+1):
    observation = env.reset()
    terminated = False
    score = 0
    while not terminated:
        env.render()
        action, _ = model.predict(observation)
        observation, reward, terminated , info = env.step(action)
        score += reward
    print('Episone:{} Score:{}'.format( episode, score))
env.close()  

Episone:1 Score:[282.75]
Episone:2 Score:[285.]
Episone:3 Score:[360.]
Episone:4 Score:[342.5]
Episone:5 Score:[262.5]
Episone:6 Score:[357.5]
Episone:7 Score:[332.5]
Episone:8 Score:[259.5]
Episone:9 Score:[342.5]
Episone:10 Score:[322.5]


# custom neural network injected in PPO model based on tria 3D environment

In [185]:
net_arch = dict(pi=[128,128,128,128], vf=[128,128,128,128])

In [186]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cpu device


In [187]:
model.learn(total_timesteps=neural_model_timesteps)

Logging to training/logs/PPO_5
-----------------------------
| time/              |      |
|    fps             | 92   |
|    iterations      | 1    |
|    time_elapsed    | 22   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 72          |
|    iterations           | 2           |
|    time_elapsed         | 56          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010003677 |
|    clip_fraction        | 0.115       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.46       |
|    explained_variance   | -0.000324   |
|    learning_rate        | 0.0003      |
|    loss                 | 885         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0101     |
|    value_loss           | 2.43e+03    |
-----------------------------------------
---

In [188]:
tria_model_path_neural = os.path.join('training','save',neural_model)
model.save(tria_model_path_neural)

In [189]:
del model

In [190]:
model = PPO.load(tria_model_path_neural, env=env)

In [191]:
evaluate_policy(model, env, n_eval_episodes=20, render=True)

(79.5875, 8.751883725804404)

In [192]:
env.close()

# create A2C network based learing with Tria 3D environment

In [193]:
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import SubprocVecEnv

In [194]:
model = A2C("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [195]:
model.learn(total_timesteps=a2c_model_timesteps)

Logging to training/logs/A2C_2
------------------------------------
| time/                 |          |
|    fps                | 72       |
|    iterations         | 100      |
|    time_elapsed       | 6        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -3.39    |
|    explained_variance | 3.45e-05 |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 60.2     |
|    value_loss         | 371      |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 76        |
|    iterations         | 200       |
|    time_elapsed       | 13        |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -3.21     |
|    explained_variance | -0.000909 |
|    learning_rate      | 0.0007    |
|    n_updates          | 199       |
|    policy_loss        | 15.4   

In [196]:
tria_model_path_a2c = os.path.join('training','save',a2c_model)
model.save(tria_model_path_a2c)

In [197]:
del model

In [198]:
model = A2C.load(tria_model_path_a2c, env=env)

In [199]:
evaluate_policy(model, env, n_eval_episodes=20, render=True)

(360.0, 0.0)

In [ ]:
env.close()

In [201]:
!tensorboard --logdir './training/logs/' --bind_all  # training_log_path

2023-03-07 20:51:36.652546: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 20:51:37.101809: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 20:51:37.101989: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 20:51:40.960059: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-